In [ ]:
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import time
import importlib
from pathlib import Path
from datetime import datetime

from numba import jit
import numpy as np
import pandas as pd

from neuropy import core

# from pyphoplacecellanalysis.General.SpikesDataframeWindow import SpikesDataframeWindow, SpikesWindowOwningMixin
# from pyphoplacecellanalysis.General.DataSeriesToSpatial import DataSeriesToSpatial
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Spike3DRaster import Spike3DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Spike3DRaster_Vedo import Spike3DRaster_Vedo

%gui qt5
# curr_epoch_name = 'maze1'
# curr_epoch = curr_active_pipeline.filtered_epochs[curr_epoch_name] # <NamedTimerange: {'name': 'maze1', 'start_end_times': array([  22.26      , 1739.15336412])};>
# curr_sess = curr_active_pipeline.filtered_sessions[curr_epoch_name]
# curr_spikes_df = curr_sess.spikes_df

from vedo import show

def trap_exc_during_debug(*args):
    # when app raises uncaught exception, print info
    print(args)


# install exception hook: without this, uncaught exception would cause application to exit
sys.excepthook = trap_exc_during_debug



In [ ]:
# Load the saved .h5 spikes dataframe for testing:
finalized_spike_df_cache_file='./pipeline_cache_store.h5'
desired_spikes_df_key = '/filtered_sessions/maze1/spikes_df'
curr_spikes_df = pd.read_hdf(finalized_spike_df_cache_file, key=desired_spikes_df_key)
curr_spikes_df

# Vedo 3D Spike Train Visualization 

In [ ]:
# Build the 3D Raster object:
spike_raster_plt = Spike3DRaster_Vedo(curr_spikes_df, window_duration=4.0, window_start_time=30.0, neuron_colors=None)


In [ ]:
spike_raster_plt._update_plots()
axes = dict(xtitle='window time', ytitle='cell_id', ztitle="")
plt = show(spike_raster_plt.glyph, __doc__, axes=axes, interactive=False, bg='k8', backend='ipyvtk') # plt: <vedo.plotter.Plotter at 0x22eb25de370>

# Pyqtgraph 3D Spike Train Visualization

In [ ]:
spike_raster_plt = Spike3DRaster(curr_spikes_df, window_duration=4.0, window_start_time=30.0, neuron_colors=None)
# spike_raster_plt = Spike3DRaster(curr_spikes_df, window_duration=0.2, window_start_time=30.0)

In [ ]:
print(f'spike_raster_plt.spikes_window.active_time_window: {spike_raster_plt.spikes_window.active_time_window}')
# spike_raster_plt.spikes_window.active_window_start_time = 50.0
spike_raster_plt.spikes_window.update_window_start(90.0)
print(f'spike_raster_plt.spikes_window.active_time_window: {spike_raster_plt.spikes_window.active_time_window}')